In [136]:
import pandas as pd
import numpy as np
import json

In [302]:
EVAL_FILE_PATH = './evaluation_2024-03-06-16-21-42.csv'
GAMEPLAY_RESULTS_PATH = './game_play_data_2024-03-06-16-21-42.json'

eval_df = pd.read_csv(EVAL_FILE_PATH)


with open(GAMEPLAY_RESULTS_PATH) as f:
    results_data = json.loads(f.read())

In [303]:
quest_results = results_data['quest_results']
quest_results

{'1': 0, '2': 0, '3': 1, '4': 0, '5': None}

In [304]:
evil_roles = ['Assassin', 'Morgana']

player_to_char = dict((val,key) for key, val in results_data['character_to_player'].items())
player_to_char

{'Player3': 'Merlin',
 'Player4': 'Percival',
 'Player1': 'Servant',
 'Player2': 'Assassin',
 'Player5': 'Morgana'}

## Failure vote rate

In [305]:
evil_players = [key for key, val in player_to_char.items() if val in evil_roles]

total_vote = 0
fail_vote = 0
for val in results_data['team_quest_result'].values():
    for p in evil_players:
        if p in val:
            if val[p] == 'failure':
                fail_vote += 1
            total_vote += 1
            
print(f"Failure rate: {fail_vote}/{total_vote}")

Failure rate: 3/4


## Quest win rate

In [306]:
evil = 0
loyal = 0
for key, val in quest_results.items():
    if val is None:
        continue
    if val == 0:
        evil += 1
    else:
        loyal += 1
        
print(f"Quest win rate: Evil: {evil}, Loyal: {loyal}")

Quest win rate: Evil: 3, Loyal: 1


## Team Selection Accuracy

In [307]:
round_leaders = results_data['round_leaders']
round_team_members = results_data['round_team_members']

total_evil = 0
evil_correct = 0

total_loyal = 0
loyal_correct = 0
for key, val in round_team_members.items():
    if len(val) == 0:
        continue
    leader = round_leaders[key]
    
    if leader in evil_players:
        total_evil += 1
        for member in val:
            role = player_to_char[member]
            if role in evil_roles:
                evil_correct += 1
                break
    else:
        total_loyal += 1
        found_evil = False
        for member in val:
            role = player_to_char[member]
            if role in evil_roles:
                found_evil = True
                break
        if not found_evil:
            loyal_correct += 1
            
print(f"Team Selection Accuracy: Evil = {evil_correct}/{total_evil}, Loyal = {loyal_correct}/{total_loyal}")

Team Selection Accuracy: Evil = 1/2, Loyal = 0/2


## Detailed analysis

In [308]:
comma_string_to_list = lambda x: [val.strip() for val in x.split(',') if val != '']

In [309]:
eval_df['intent_selection'] = eval_df['intent_selection'].astype(str).apply(comma_string_to_list)
eval_df['intent_selection_mod'] = eval_df['intent_selection_mod'].astype(str).apply(comma_string_to_list)
eval_df['intent_fol_think_1'] = eval_df['intent_fol_think_1'].astype(str).apply(comma_string_to_list)
eval_df['intent_fol_speech_1'] = eval_df['intent_fol_speech_1'].astype(str).apply(comma_string_to_list)
eval_df['intent_fol_think_2'] = eval_df['intent_fol_think_2'].astype(str).apply(comma_string_to_list)
eval_df['intent_fol_speech_2'] = eval_df['intent_fol_speech_2'].astype(str).apply(comma_string_to_list)
eval_df['round_quest_result'] = eval_df['round'].apply(lambda x: 'success' if quest_results[str(x)] == 1 else 'failure')

eval_df.head()

,player,round,intent,intent_selection,intent_fol_think_1,intent_fol_speech_1,intent_mod,intent_selection_mod,intent_fol_think_2,intent_fol_speech_2,comments,round_quest_result
0,Player1,1,['Strongly convince the team leader to choose ...,[0],[5],[5],NaN,[nan],[5],[5],team leader try to convince himself,failure
1,Player2,1,['counter the team proposal because you are go...,[0],[1],[1],NaN,[nan],[1],[1],wrong intent understanding,failure
2,Player3,1,"['Support team proposal', 'Support one loyal p...","[1, 1, 1]","[5, 5, 5]","[5, 5, 4]",NaN,[nan],"[5, 5, 5]","[5, 5, 5]",NaN,failure
3,Player4,1,['Question the leader why they selected a part...,"[1, 1, 1]","[5, 5, 5]","[5, 5, 5]",['Stay hidden in discussions and act like a Lo...,[1],[5],[5],NaN,failure
4,Player5,1,"['Pretend to be Merlin', 'Stay hidden in discu...","[1, 0, 1]","[5, 5, 5]","[1, 4, 5]",NaN,[nan],"[5, 5, 5]","[1, 4, 5]",NaN,failure


In [310]:
def find_player_side(player_name):
    if player_to_char[player_name] in evil_roles:
        return 'evil'
    return 'loyal'

In [311]:
eval_df['side'] = eval_df['player'].apply(lambda x: find_player_side(x))
eval_df.head()

,player,round,intent,intent_selection,intent_fol_think_1,intent_fol_speech_1,intent_mod,intent_selection_mod,intent_fol_think_2,intent_fol_speech_2,comments,round_quest_result,side
0,Player1,1,['Strongly convince the team leader to choose ...,[0],[5],[5],NaN,[nan],[5],[5],team leader try to convince himself,failure,loyal
1,Player2,1,['counter the team proposal because you are go...,[0],[1],[1],NaN,[nan],[1],[1],wrong intent understanding,failure,evil
2,Player3,1,"['Support team proposal', 'Support one loyal p...","[1, 1, 1]","[5, 5, 5]","[5, 5, 4]",NaN,[nan],"[5, 5, 5]","[5, 5, 5]",NaN,failure,loyal
3,Player4,1,['Question the leader why they selected a part...,"[1, 1, 1]","[5, 5, 5]","[5, 5, 5]",['Stay hidden in discussions and act like a Lo...,[1],[5],[5],NaN,failure,loyal
4,Player5,1,"['Pretend to be Merlin', 'Stay hidden in discu...","[1, 0, 1]","[5, 5, 5]","[1, 4, 5]",NaN,[nan],"[5, 5, 5]","[1, 4, 5]",NaN,failure,evil


In [312]:
eval_df_form_cnt = eval_df[['player', 'side', 'round', 'intent', 'intent_selection', 'intent_fol_think_1', 'intent_fol_speech_1', 'round_quest_result']].copy()
eval_df_ref_cnt = eval_df[['player', 'side', 'round', 'intent', 'intent_mod', 'intent_selection', 'intent_selection_mod', 'intent_fol_think_2', 'intent_fol_speech_2', 'round_quest_result']].copy()

In [313]:
eval_df_form_cnt.head()

,player,side,round,intent,intent_selection,intent_fol_think_1,intent_fol_speech_1,round_quest_result
0,Player1,loyal,1,['Strongly convince the team leader to choose ...,[0],[5],[5],failure
1,Player2,evil,1,['counter the team proposal because you are go...,[0],[1],[1],failure
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...","[1, 1, 1]","[5, 5, 5]","[5, 5, 4]",failure
3,Player4,loyal,1,['Question the leader why they selected a part...,"[1, 1, 1]","[5, 5, 5]","[5, 5, 5]",failure
4,Player5,evil,1,"['Pretend to be Merlin', 'Stay hidden in discu...","[1, 0, 1]","[5, 5, 5]","[1, 4, 5]",failure


In [314]:
eval_df_ref_cnt.head()

,player,side,round,intent,intent_mod,intent_selection,intent_selection_mod,intent_fol_think_2,intent_fol_speech_2,round_quest_result
0,Player1,loyal,1,['Strongly convince the team leader to choose ...,NaN,[0],[nan],[5],[5],failure
1,Player2,evil,1,['counter the team proposal because you are go...,NaN,[0],[nan],[1],[1],failure
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...",NaN,"[1, 1, 1]",[nan],"[5, 5, 5]","[5, 5, 5]",failure
3,Player4,loyal,1,['Question the leader why they selected a part...,['Stay hidden in discussions and act like a Lo...,"[1, 1, 1]",[1],[5],[5],failure
4,Player5,evil,1,"['Pretend to be Merlin', 'Stay hidden in discu...",NaN,"[1, 0, 1]",[nan],"[5, 5, 5]","[1, 4, 5]",failure


## Processing dfs for analysis (splitting values into separate rows)

#### Formulation contemplation

In [315]:
eval_df_form_cnt_denorm = eval_df_form_cnt.explode(['intent_selection', 'intent_fol_think_1', 'intent_fol_speech_1'])
eval_df_form_cnt_denorm.head(10)

,player,side,round,intent,intent_selection,intent_fol_think_1,intent_fol_speech_1,round_quest_result
0,Player1,loyal,1,['Strongly convince the team leader to choose ...,0,5,5,failure
1,Player2,evil,1,['counter the team proposal because you are go...,0,1,1,failure
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...",1,5,5,failure
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...",1,5,5,failure
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...",1,5,4,failure
3,Player4,loyal,1,['Question the leader why they selected a part...,1,5,5,failure
3,Player4,loyal,1,['Question the leader why they selected a part...,1,5,5,failure
3,Player4,loyal,1,['Question the leader why they selected a part...,1,5,5,failure
4,Player5,evil,1,"['Pretend to be Merlin', 'Stay hidden in discu...",1,5,1,failure
4,Player5,evil,1,"['Pretend to be Merlin', 'Stay hidden in discu...",0,5,4,failure


In [316]:
eval_df_form_cnt_denorm['intent_selection'] = eval_df_form_cnt_denorm['intent_selection'].astype(int)
eval_df_form_cnt_denorm['intent_fol_think_1'] = eval_df_form_cnt_denorm['intent_fol_think_1'].astype(int)
eval_df_form_cnt_denorm['intent_fol_speech_1'] = eval_df_form_cnt_denorm['intent_fol_speech_1'].astype(int)

In [317]:
eval_df_ref_cnt.head()

,player,side,round,intent,intent_mod,intent_selection,intent_selection_mod,intent_fol_think_2,intent_fol_speech_2,round_quest_result
0,Player1,loyal,1,['Strongly convince the team leader to choose ...,NaN,[0],[nan],[5],[5],failure
1,Player2,evil,1,['counter the team proposal because you are go...,NaN,[0],[nan],[1],[1],failure
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...",NaN,"[1, 1, 1]",[nan],"[5, 5, 5]","[5, 5, 5]",failure
3,Player4,loyal,1,['Question the leader why they selected a part...,['Stay hidden in discussions and act like a Lo...,"[1, 1, 1]",[1],[5],[5],failure
4,Player5,evil,1,"['Pretend to be Merlin', 'Stay hidden in discu...",NaN,"[1, 0, 1]",[nan],"[5, 5, 5]","[1, 4, 5]",failure


#### Refinement Contemplation

In [318]:
def update_intent_selection_vals(intent_mod, intent_sel, intent_sel_mod):
    if intent_mod is not np.NaN:
        return intent_sel_mod
    return intent_sel

In [319]:
eval_df_ref_cnt['intent_selection_final'] = eval_df_ref_cnt.apply(lambda row: update_intent_selection_vals(row.intent_mod, row.intent_selection, row.intent_selection_mod), axis=1)
eval_df_ref_cnt.head()

,player,side,round,intent,intent_mod,intent_selection,intent_selection_mod,intent_fol_think_2,intent_fol_speech_2,round_quest_result,intent_selection_final
0,Player1,loyal,1,['Strongly convince the team leader to choose ...,NaN,[0],[nan],[5],[5],failure,[0]
1,Player2,evil,1,['counter the team proposal because you are go...,NaN,[0],[nan],[1],[1],failure,[0]
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...",NaN,"[1, 1, 1]",[nan],"[5, 5, 5]","[5, 5, 5]",failure,"[1, 1, 1]"
3,Player4,loyal,1,['Question the leader why they selected a part...,['Stay hidden in discussions and act like a Lo...,"[1, 1, 1]",[1],[5],[5],failure,[1]
4,Player5,evil,1,"['Pretend to be Merlin', 'Stay hidden in discu...",NaN,"[1, 0, 1]",[nan],"[5, 5, 5]","[1, 4, 5]",failure,"[1, 0, 1]"


In [320]:
eval_df_ref_cnt.drop(['intent_selection','intent_selection_mod'], axis=1, inplace=True)
eval_df_ref_cnt.head()

,player,side,round,intent,intent_mod,intent_fol_think_2,intent_fol_speech_2,round_quest_result,intent_selection_final
0,Player1,loyal,1,['Strongly convince the team leader to choose ...,NaN,[5],[5],failure,[0]
1,Player2,evil,1,['counter the team proposal because you are go...,NaN,[1],[1],failure,[0]
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...",NaN,"[5, 5, 5]","[5, 5, 5]",failure,"[1, 1, 1]"
3,Player4,loyal,1,['Question the leader why they selected a part...,['Stay hidden in discussions and act like a Lo...,[5],[5],failure,[1]
4,Player5,evil,1,"['Pretend to be Merlin', 'Stay hidden in discu...",NaN,"[5, 5, 5]","[1, 4, 5]",failure,"[1, 0, 1]"


In [321]:
eval_df_ref_cnt_denorm = eval_df_ref_cnt.explode(['intent_selection_final', 'intent_fol_think_2', 'intent_fol_speech_2'])
eval_df_ref_cnt_denorm.head(10)

,player,side,round,intent,intent_mod,intent_fol_think_2,intent_fol_speech_2,round_quest_result,intent_selection_final
0,Player1,loyal,1,['Strongly convince the team leader to choose ...,NaN,5,5,failure,0
1,Player2,evil,1,['counter the team proposal because you are go...,NaN,1,1,failure,0
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...",NaN,5,5,failure,1
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...",NaN,5,5,failure,1
2,Player3,loyal,1,"['Support team proposal', 'Support one loyal p...",NaN,5,5,failure,1
3,Player4,loyal,1,['Question the leader why they selected a part...,['Stay hidden in discussions and act like a Lo...,5,5,failure,1
4,Player5,evil,1,"['Pretend to be Merlin', 'Stay hidden in discu...",NaN,5,1,failure,1
4,Player5,evil,1,"['Pretend to be Merlin', 'Stay hidden in discu...",NaN,5,4,failure,0
4,Player5,evil,1,"['Pretend to be Merlin', 'Stay hidden in discu...",NaN,5,5,failure,1
5,Player2,evil,2,['Stay hidden in discussions and act like a Lo...,NaN,5,1,failure,1


In [322]:
eval_df_ref_cnt_denorm['intent_selection_final'] = eval_df_ref_cnt_denorm['intent_selection_final'].astype(int)
eval_df_ref_cnt_denorm['intent_fol_think_2'] = eval_df_ref_cnt_denorm['intent_fol_think_2'].astype(int)
eval_df_ref_cnt_denorm['intent_fol_speech_2'] = eval_df_ref_cnt_denorm['intent_fol_speech_2'].astype(int)

## Metrics

### Calculating stats

In [323]:
metrics_form_cnt_think = eval_df_form_cnt_denorm.groupby(['side', 'intent_selection', 'intent_fol_think_1']).size().reset_index(name='count')
print(metrics_form_cnt_think)

    side  intent_selection  intent_fol_think_1  count
0   evil                 0                   1      1
1   evil                 0                   5      6
2   evil                 1                   5     29
3  loyal                 0                   1      2
4  loyal                 0                   5     10
5  loyal                 1                   5     41


In [324]:
metrics_form_cnt_think_rnd = eval_df_form_cnt_denorm.groupby(['side', 'intent_selection', 'intent_fol_think_1', 'round', 'round_quest_result']).size().reset_index(name='count')

In [325]:
metrics_form_cnt_speech = eval_df_form_cnt_denorm.groupby(['side', 'intent_selection', 'intent_fol_speech_1']).size().reset_index(name='count')
print(metrics_form_cnt_speech)

     side  intent_selection  intent_fol_speech_1  count
0    evil                 0                    1      1
1    evil                 0                    4      1
2    evil                 0                    5      5
3    evil                 1                    1      1
4    evil                 1                    2      2
5    evil                 1                    3      2
6    evil                 1                    4      3
7    evil                 1                    5     21
8   loyal                 0                    4      1
9   loyal                 0                    5     11
10  loyal                 1                    3      7
11  loyal                 1                    4      5
12  loyal                 1                    5     29


In [326]:
metrics_form_cnt_speech_rnd = eval_df_form_cnt_denorm.groupby(['side', 'intent_selection', 'intent_fol_speech_1', 'round', 'round_quest_result']).size().reset_index(name='count')

In [327]:
metrics_ref_cnt_think = eval_df_ref_cnt_denorm.groupby(['side', 'intent_selection_final', 'intent_fol_think_2']).size().reset_index(name='count')
print(metrics_ref_cnt_think)

    side  intent_selection_final  intent_fol_think_2  count
0   evil                       0                   1      1
1   evil                       0                   5      5
2   evil                       1                   1      4
3   evil                       1                   3      1
4   evil                       1                   5     24
5  loyal                       0                   1      2
6  loyal                       0                   5     10
7  loyal                       1                   1      3
8  loyal                       1                   5     35


In [328]:
metrics_ref_cnt_think_rnd = eval_df_ref_cnt_denorm.groupby(['side', 'intent_selection_final', 'intent_fol_think_2', 'round', 'round_quest_result']).size().reset_index(name='count')

In [329]:
metrics_ref_cnt_speech = eval_df_ref_cnt_denorm.groupby(['side', 'intent_selection_final', 'intent_fol_speech_2']).size().reset_index(name='count')
print(metrics_ref_cnt_speech)

     side  intent_selection_final  intent_fol_speech_2  count
0    evil                       0                    1      1
1    evil                       0                    4      2
2    evil                       0                    5      3
3    evil                       1                    1     10
4    evil                       1                    3      2
5    evil                       1                    4      2
6    evil                       1                    5     15
7   loyal                       0                    4      1
8   loyal                       0                    5     11
9   loyal                       1                    1      3
10  loyal                       1                    3      9
11  loyal                       1                    4      2
12  loyal                       1                    5     24


In [330]:
metrics_ref_cnt_speech_rnd = eval_df_ref_cnt_denorm.groupby(['side', 'intent_selection_final', 'intent_fol_speech_2', 'round', 'round_quest_result']).size().reset_index(name='count')

## Final metrics calc

In [331]:
metrics_form_cnt_think = metrics_form_cnt_think.rename(columns={'intent_fol_think_1': 'intent_fol_think'})
metrics_ref_cnt_think = metrics_ref_cnt_think.rename(columns={'intent_fol_think_2': 'intent_fol_think', 'intent_selection_final': 'intent_selection'})
metrics_form_cnt_speech = metrics_form_cnt_speech.rename(columns={'intent_fol_speech_1': 'intent_fol_speech'})
metrics_ref_cnt_speech = metrics_ref_cnt_speech.rename(columns={'intent_fol_speech_2': 'intent_fol_speech', 'intent_selection_final': 'intent_selection'})

In [332]:
metrics_form_cnt_think.head()

,side,intent_selection,intent_fol_think,count
0,evil,0,1,1
1,evil,0,5,6
2,evil,1,5,29
3,loyal,0,1,2
4,loyal,0,5,10


In [333]:
metrics_form_cnt_think_rnd = metrics_form_cnt_think_rnd.rename(columns={'intent_fol_think_1': 'intent_fol_think'})
metrics_ref_cnt_think_rnd = metrics_ref_cnt_think_rnd.rename(columns={'intent_fol_think_2': 'intent_fol_think', 'intent_selection_final': 'intent_selection'})
metrics_form_cnt_speech_rnd = metrics_form_cnt_speech_rnd.rename(columns={'intent_fol_speech_1': 'intent_fol_speech'})
metrics_ref_cnt_speech_rnd = metrics_ref_cnt_speech_rnd.rename(columns={'intent_fol_speech_2': 'intent_fol_speech', 'intent_selection_final': 'intent_selection'})

In [334]:
metrics_think = pd.concat([metrics_form_cnt_think, metrics_ref_cnt_think])
metrics_think.head()

,side,intent_selection,intent_fol_think,count
0,evil,0,1,1
1,evil,0,5,6
2,evil,1,5,29
3,loyal,0,1,2
4,loyal,0,5,10


In [335]:
metrics_think_rnd = pd.concat([metrics_form_cnt_think_rnd, metrics_ref_cnt_think_rnd])
metrics_think_rnd.head()

,side,intent_selection,intent_fol_think,round,round_quest_result,count
0,evil,0,1,1,failure,1
1,evil,0,5,1,failure,1
2,evil,0,5,2,failure,2
3,evil,0,5,3,success,1
4,evil,0,5,4,failure,2


In [336]:
metrics_speech = pd.concat([metrics_form_cnt_speech, metrics_ref_cnt_speech])
metrics_speech.head()

,side,intent_selection,intent_fol_speech,count
0,evil,0,1,1
1,evil,0,4,1
2,evil,0,5,5
3,evil,1,1,1
4,evil,1,2,2


In [337]:
metrics_speech_rnd = pd.concat([metrics_form_cnt_speech_rnd, metrics_ref_cnt_speech_rnd])
metrics_speech_rnd

,side,intent_selection,intent_fol_speech,round,round_quest_result,count
0,evil,0,1,1,failure,1
1,evil,0,4,1,failure,1
2,evil,0,5,2,failure,2
3,evil,0,5,3,success,1
4,evil,0,5,4,failure,2
5,evil,1,1,1,failure,1
6,evil,1,2,3,success,1
7,evil,1,2,4,failure,1
8,evil,1,3,4,failure,2
9,evil,1,4,3,success,3


In [338]:
metrics_speech_rnd = metrics_speech_rnd.rename(columns={'intent_fol_speech': 'intent_fol'})
metrics_think_rnd = metrics_think_rnd.rename(columns={'intent_fol_think': 'intent_fol'})

metrics_rnd = pd.concat([metrics_think_rnd, metrics_speech_rnd])
metrics_rnd.head()

,side,intent_selection,intent_fol,round,round_quest_result,count
0,evil,0,1,1,failure,1
1,evil,0,5,1,failure,1
2,evil,0,5,2,failure,2
3,evil,0,5,3,success,1
4,evil,0,5,4,failure,2


In [339]:
total_think = metrics_think.groupby(['side', 'intent_selection'])['count'].sum().reset_index(name='count')
print(total_think)

    side  intent_selection  count
0   evil                 0     13
1   evil                 1     58
2  loyal                 0     24
3  loyal                 1     79


In [340]:
total_speech = metrics_speech.groupby(['side', 'intent_selection'])['count'].sum().reset_index(name='count')
print(total_speech)

    side  intent_selection  count
0   evil                 0     13
1   evil                 1     58
2  loyal                 0     24
3  loyal                 1     79


In [341]:
greater_than_3_think = metrics_think[metrics_think['intent_fol_think'] >= 3].groupby(['side', 'intent_selection'])['count'].sum().reset_index(name='count')
print(greater_than_3_think)

    side  intent_selection  count
0   evil                 0     11
1   evil                 1     54
2  loyal                 0     20
3  loyal                 1     76


In [342]:
greater_than_3_speech = metrics_speech[metrics_speech['intent_fol_speech'] >= 3].groupby(['side', 'intent_selection'])['count'].sum().reset_index(name='count')
print(greater_than_3_speech)

    side  intent_selection  count
0   evil                 0     11
1   evil                 1     45
2  loyal                 0     24
3  loyal                 1     76


In [343]:
think_5 = metrics_think[metrics_think['intent_fol_think'] == 5].groupby(['side', 'intent_selection'])['count'].sum().reset_index(name='count')
print(think_5)

    side  intent_selection  count
0   evil                 0     11
1   evil                 1     53
2  loyal                 0     20
3  loyal                 1     76


In [344]:
speech_5 = metrics_speech[metrics_speech['intent_fol_speech'] == 5].groupby(['side', 'intent_selection'])['count'].sum().reset_index(name='count')
print(speech_5)

    side  intent_selection  count
0   evil                 0      8
1   evil                 1     36
2  loyal                 0     22
3  loyal                 1     53


In [345]:
keys = [('evil', 0), ('evil', 1), ('loyal', 0), ('loyal', 1)]

In [346]:
final_total_metrics_gt_3 = {'evil': {}, 'loyal': {}}

final_total_metrics_5 = {'evil': {}, 'loyal': {}}

In [347]:
for side, is_reas in keys:
    total_think_count = total_think[(total_think['side'] == side) & (total_think['intent_selection'] == is_reas)]['count'].values[0]
    total_speech_count = total_speech[(total_speech['side'] == side) & (total_speech['intent_selection'] == is_reas)]['count'].values[0]
    
    greater_than_3_think_count = greater_than_3_think[(greater_than_3_think['side'] == side) & (greater_than_3_think['intent_selection'] == is_reas)]['count'].values[0]
    greater_than_3_speech_count = greater_than_3_speech[(greater_than_3_speech['side'] == side) & (greater_than_3_speech['intent_selection'] == is_reas)]['count'].values[0]
    
    think_5_count = think_5[(think_5['side'] == side) & (think_5['intent_selection'] == is_reas)]['count'].values[0]

    speech_5_count = speech_5[(speech_5['side'] == side) & (speech_5['intent_selection'] == is_reas)]['count'].values[0]
    
    intent_reas = 'reasonable' if is_reas == 1 else 'unreasonable'
    
    final_total_metrics_gt_3[side][intent_reas] = {
        'following': {'think': greater_than_3_think_count, 'speak': greater_than_3_speech_count},
        'not_following': {'think': total_think_count - greater_than_3_think_count, 'speak': total_speech_count - greater_than_3_speech_count}
    }
    
    final_total_metrics_5[side][intent_reas] = {
        'following': {'think': think_5_count, 'speak': speech_5_count},
        'not_following': {'think': total_think_count - think_5_count, 'speak': total_speech_count - speech_5_count}
    }

In [348]:
final_total_metrics_gt_3

{'evil': {'unreasonable': {'following': {'think': 11, 'speak': 11},
   'not_following': {'think': 2, 'speak': 2}},
  'reasonable': {'following': {'think': 54, 'speak': 45},
   'not_following': {'think': 4, 'speak': 13}}},
 'loyal': {'unreasonable': {'following': {'think': 20, 'speak': 24},
   'not_following': {'think': 4, 'speak': 0}},
  'reasonable': {'following': {'think': 76, 'speak': 76},
   'not_following': {'think': 3, 'speak': 3}}}}

In [349]:
final_total_metrics_5

{'evil': {'unreasonable': {'following': {'think': 11, 'speak': 8},
   'not_following': {'think': 2, 'speak': 5}},
  'reasonable': {'following': {'think': 53, 'speak': 36},
   'not_following': {'think': 5, 'speak': 22}}},
 'loyal': {'unreasonable': {'following': {'think': 20, 'speak': 22},
   'not_following': {'think': 4, 'speak': 2}},
  'reasonable': {'following': {'think': 76, 'speak': 53},
   'not_following': {'think': 3, 'speak': 26}}}}

In [350]:
print(total_think)

    side  intent_selection  count
0   evil                 0     13
1   evil                 1     58
2  loyal                 0     24
3  loyal                 1     79


In [351]:
print(total_speech)

    side  intent_selection  count
0   evil                 0     13
1   evil                 1     58
2  loyal                 0     24
3  loyal                 1     79


### Round metrics

In [352]:
total_rnd = metrics_rnd.groupby(['side', 'intent_selection', 'round', 'round_quest_result'])['count'].sum().reset_index(name='count')
print(total_rnd)

     side  intent_selection  round round_quest_result  count
0    evil                 0      1            failure      8
1    evil                 0      2            failure     10
2    evil                 0      3            success      4
3    evil                 0      4            failure      4
4    evil                 1      1            failure      8
5    evil                 1      2            failure     36
6    evil                 1      3            success     44
7    evil                 1      4            failure     28
8   loyal                 0      1            failure      4
9   loyal                 0      2            failure     24
10  loyal                 0      3            success     12
11  loyal                 0      4            failure      8
12  loyal                 1      1            failure     20
13  loyal                 1      2            failure     40
14  loyal                 1      3            success     46
15  loyal               

In [353]:
total_rnd['count'].sum()

348

In [354]:
rnd_5 = metrics_rnd[metrics_rnd['intent_fol'] == 5].groupby(['side', 'intent_selection', 'round', 'round_quest_result'])['count'].sum().reset_index(name='count')
print(rnd_5)

     side  intent_selection  round round_quest_result  count
0    evil                 0      1            failure      2
1    evil                 0      2            failure      9
2    evil                 0      3            success      4
3    evil                 0      4            failure      4
4    evil                 1      1            failure      6
5    evil                 1      2            failure     35
6    evil                 1      3            success     28
7    evil                 1      4            failure     20
8   loyal                 0      1            failure      4
9   loyal                 0      2            failure     22
10  loyal                 0      3            success      8
11  loyal                 0      4            failure      8
12  loyal                 1      1            failure     19
13  loyal                 1      2            failure     35
14  loyal                 1      3            success     39
15  loyal               

In [355]:
rnd_5['count'].sum()

279

In [356]:
rnd_5.head()

,side,intent_selection,round,round_quest_result,count
0,evil,0,1,failure,2
1,evil,0,2,failure,9
2,evil,0,3,success,4
3,evil,0,4,failure,4
4,evil,1,1,failure,6


In [357]:
rnd_5['rnd'] = rnd_5['round'].astype(int)
rnd_5['sd'] = rnd_5['side'].astype(str)

In [358]:
round_metrics_dict = {}

In [359]:
def round_metrics(side, intent_sel, rnd, count):
    if side not in round_metrics_dict:
        round_metrics_dict[side] = {}
    if rnd not in round_metrics_dict[side]:
        round_metrics_dict[side][rnd] = {}
    round_metrics_dict[side][rnd][intent_sel] = count
    

In [360]:
for index, row in rnd_5.iterrows():
    round_metrics(row['sd'], row['intent_selection'], row['rnd'], row['count'])


In [361]:
round_metrics_dict

{'evil': {1: {0: 2, 1: 6},
  2: {0: 9, 1: 35},
  3: {0: 4, 1: 28},
  4: {0: 4, 1: 20}},
 'loyal': {1: {0: 4, 1: 19},
  2: {0: 22, 1: 35},
  3: {0: 8, 1: 39},
  4: {0: 8, 1: 36}}}

In [362]:
quest_results

{'1': 0, '2': 0, '3': 1, '4': 0, '5': None}

In [369]:
final_round_metrics = {}

In [370]:
for key in quest_results.keys():
    if quest_results[key] is None:
        continue
    int_key = int(key)
    result = 'success' if quest_results[key] == 1 else 'failure'
    
    evil_reas = 0
    evil_unreas = 0
    loyal_reas = 0
    loyal_unreas = 0
    
    if 0 in round_metrics_dict['evil'][int_key]:
        evil_unreas = round_metrics_dict['evil'][int_key][0]
        
    if 1 in round_metrics_dict['evil'][int_key]:
        evil_reas = round_metrics_dict['evil'][int_key][1]
        
    if 0 in round_metrics_dict['loyal'][int_key]:
        loyal_unreas = round_metrics_dict['loyal'][int_key][0]
        
    if 1 in round_metrics_dict['loyal'][int_key]:
        loyal_reas = round_metrics_dict['loyal'][int_key][1]
        
    total_evil_intents = evil_unreas + evil_reas
    total_loyal_intents = loyal_unreas + loyal_reas
    
    evil_reas_per = evil_reas/total_evil_intents
    evil_unreas_per = evil_unreas/total_evil_intents
    
    loyal_reas_per = loyal_reas/total_loyal_intents
    loyal_unreas_per = loyal_unreas/total_loyal_intents
    
    final_round_metrics[int_key] = {
        'result': result, 
        'evil_reasonable': f"{evil_reas_per} ({evil_reas})", 
        'evil_unreasonable': f"{evil_unreas_per} ({evil_unreas})", 
        'loyal_reasonable': f"{loyal_reas_per} ({loyal_reas})", 
        'loyal_unreasonable': f"{loyal_unreas_per} ({loyal_unreas})"
    }

In [371]:
final_round_metrics

{1: {'result': 'failure',
  'evil_reasonable': '0.75 (6)',
  'evil_unreasonable': '0.25 (2)',
  'loyal_reasonable': '0.8260869565217391 (19)',
  'loyal_unreasonable': '0.17391304347826086 (4)'},
 2: {'result': 'failure',
  'evil_reasonable': '0.7954545454545454 (35)',
  'evil_unreasonable': '0.20454545454545456 (9)',
  'loyal_reasonable': '0.6140350877192983 (35)',
  'loyal_unreasonable': '0.38596491228070173 (22)'},
 3: {'result': 'success',
  'evil_reasonable': '0.875 (28)',
  'evil_unreasonable': '0.125 (4)',
  'loyal_reasonable': '0.8297872340425532 (39)',
  'loyal_unreasonable': '0.1702127659574468 (8)'},
 4: {'result': 'failure',
  'evil_reasonable': '0.8333333333333334 (20)',
  'evil_unreasonable': '0.16666666666666666 (4)',
  'loyal_reasonable': '0.8181818181818182 (36)',
  'loyal_unreasonable': '0.18181818181818182 (8)'}}